### Bayesian Example Using Markov Chain Monte Carlo  

In [1]:
using namespace RooStats;

In [2]:
RooStats::MCMCInterval * interval = nullptr;
RooStats::MCMCIntervalPlot * plot = nullptr; 
RooStats::ProposalFunction * proposalFunc = nullptr;

First part is just to access the workspace file and retrieve the model and the data 

In [3]:
TString fileName = "~/Downloads/dimuon_13TeV_decJamb2015_BWCB_workspace_5000.root";
//TString fileName = "HiggsModel.root";
TString workspaceName = "dimuon_13TeV_decJamb2015_BWCB";
TString modelConfigName = "mc";
TString dataName = "data";
TString integrationType = "";  
RooAbsReal::defaultIntegratorConfig()->method1D().setLabel("RooAdaptiveGaussKronrodIntegrator1D");


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



In [4]:
auto file = TFile::Open(fileName);

Warning in <TClass::Init>: no dictionary for class ZPrimeMuonBkgPdf is available
Warning in <TClass::Init>: no dictionary for class PowFunc is available
Warning in <TClass::Init>: no dictionary for class ZPrimeMuonAccEffFunc is available
Warning in <TClass::Init>: no dictionary for class ZPrimeMuonResFunc is available


In [ ]:
auto w =  (RooWorkspace*) file->Get(workspaceName);
w->Print();
auto mc = (RooStats::ModelConfig*) w->obj(modelConfigName);
auto  data = w->data(dataName);
auto poi = (RooRealVar*) mc->GetParametersOfInterest()->first();  // get POI 

[#1] INFO:ObjectHandling -- RooWorkspace::CodeRepo::compileClasses() creating code export directory .wscode.3648b4b4-eed8-11e5-9717-828fd380beef.dimuon_13TeV_decJamb2015_BWCB to extract coded embedded in workspace
[#1] INFO:ObjectHandling -- RooWorkspace::CodeRepo::compileClasses() Extracting declaration code of class PowFunc, file .wscode.3648b4b4-eed8-11e5-9717-828fd380beef.dimuon_13TeV_decJamb2015_BWCB/PowFunc.h
[#1] INFO:ObjectHandling -- RooWorkspace::CodeRepo::compileClasses() Extracting implementation code of class PowFunc, file .wscode.3648b4b4-eed8-11e5-9717-828fd380beef.dimuon_13TeV_decJamb2015_BWCB/PowFunc.cxx
[#1] INFO:ObjectHandling -- RooWorkspace::CodeRepo::compileClasses() Compiling code unit PowFunc to define class PowFunc


Info in <TMacOSXSystem::ACLiC>: creating shared library /Users/moneta/root/tutorials/DESY2016/.wscode.3648b4b4-eed8-11e5-9717-828fd380beef.dimuon_13TeV_decJamb2015_BWCB/PowFunc_cxx.so


[#0] ERROR:ObjectHandling -- RooWorkspace::CodeRepo::compileClasses() ERROR compiling class PowFunc, to fix this you can do the following: 
  1) Fix extracted source code files in directory .wscode.3648b4b4-eed8-11e5-9717-828fd380beef.dimuon_13TeV_decJamb2015_BWCB/
  2) In clean ROOT session compiled fixed classes by hand using '.x .wscode.3648b4b4-eed8-11e5-9717-828fd380beef.dimuon_13TeV_decJamb2015_BWCB/ClassName.cxx+'
  3) Reopen file with RooWorkspace with broken source code in UPDATE mode. Access RooWorkspace to force loading of class
     Broken instances in workspace will _not_ be compiled, instead precompiled fixed instances will be used.
  4) Reimport fixed code in workspace using 'RooWorkspace::importClassCode("*",kTRUE)' method, Write() updated workspace to file and close file
  5) Reopen file in clean ROOT session to confirm that problems are fixed


ld: can't link with bundle (MH_BUNDLE) only dylibs (MH_DYLIB) file '/Library/Python/2.7/site-packages/zmq/backend/cython/message.so' for architecture x86_64
clang: error: linker command failed with exit code 1 (use -v to see invocation)
Error in <ACLiC>: Compilation failed!
Error in <TBufferFile::ReadObject>: trying to read an emulated class (ZPrimeMuonBkgPdf) to store in a compiled pointer (TObject)
Error in <TBufferFile::ReadObject>: trying to read an emulated class (PowFunc) to store in a compiled pointer (TObject)

 *** Break *** segmentation violation


[<unknown binary>]
[<unknown binary>]
[/Users/moneta/root/git/master-build/lib/libRIO.so] TStreamerInfoActions::GenericReadAction(TBuffer&, void*, TStreamerInfoActions::TConfiguration const*) /Users/moneta/root/git/root/io/io/src/TStreamerInfoActions.cxx:167
[/Users/moneta/root/git/master-build/lib/libRIO.so] TBufferFile::ApplySequence(TStreamerInfoActions::TActionSequence const&, void*) /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/iterator:1190
[/Users/moneta/root/git/master-build/lib/libRIO.so] TBufferFile::ReadClassBuffer(TClass const*, void*, TClass const*) /Users/moneta/root/git/root/io/io/src/TBufferFile.cxx:3895
[/Users/moneta/root/git/master-build/lib/libRooFitCore.so] RooAbsArg::Streamer(TBuffer&) /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/deque:2561


In [ ]:
RooStats::MCMCCalculator mcmcCalc(*data,*mc);

Set the intervar Confidence Level

In [ ]:
mcmcCalc.SetConfidenceLevel(0.95);

Set the type of interval

In [ ]:
 mcmcCalc.SetLeftSideTailFraction(0.0);

In [ ]:
proposalFunc = new RooStats::SequentialProposal(0.1);
mcmcCalc.SetProposalFunction(*proposalFunc);

In [ ]:
mcmcCalc.SetNumIters(1000000);         // Metropolis-Hastings algorithm iterations
mcmcCalc.SetNumBurnInSteps(100);       // first N steps to be ignored as burn-in

In [ ]:
interval = mcmcCalc.GetInterval();

In [ ]:
cout << 100*mcmcCalc.ConfidenceLevel() << "% interval is : ["<< interval->LowerLimit(*poi) << ", "<< interval->UpperLimit(*poi) <<"] "<<endl;

In [ ]:
plot = new RooStats::MCMCIntervalPlot(*interval);

In [ ]:
plot->Draw("HIST"); gPad->Draw();